# Get SP500 data from Yahoo Finance and usd to euro conversion rate


In [8]:
import yfinance as yf
import pandas as pd
import plotly.express as px

In [15]:
start_date = '2023-01-01'
end_date = '2025-04-28'

# Download SP500 data
sp500_data = yf.download('^GSPC', start=start_date, end=end_date, interval='1d')
# Calculate the relative daily difference for each column
sp500_data_diff = sp500_data.pct_change()
sp500_data_diff

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
Date,,,,,
2023-01-03,NaN,NaN,NaN,NaN,NaN
2023-01-04,0.007539,-0.001367,0.005651,-0.003356,0.114909
2023-01-05,-0.011646,-0.008629,-0.003499,-0.000161,-0.117948
2023-01-06,0.022841,0.017306,0.001878,-0.004263,0.007734
2023-01-09,-0.000768,0.011361,0.021226,0.022872,0.098943
...,...,...,...,...,...
2025-04-21,-0.023567,-0.017899,-0.029293,-0.013667,-0.103617
2025-04-22,0.025117,0.014651,0.020786,-0.004829,0.104253


In [18]:
# Download USD to Euro conversion rate data
usd_to_euro_data = yf.download('USDEUR=X', start=start_date, end=end_date, interval='1d')
usd_to_euro_data_diff = usd_to_euro_data.pct_change()
# Extract daily close prices
usd_to_euro_data_diff

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,USDEUR=X,USDEUR=X,USDEUR=X,USDEUR=X,USDEUR=X
Date,,,,,
2023-01-02,NaN,NaN,NaN,NaN,NaN
2023-01-03,0.002999,0.012518,0.002785,0.002999,NaN
2023-01-04,0.012407,-0.002315,0.004807,0.012407,NaN
2023-01-05,-0.005611,0.002890,-0.000106,-0.005611,NaN
2023-01-06,0.007997,0.002892,0.000978,0.007997,NaN
...,...,...,...,...,...
2025-04-17,-0.009136,-0.004156,-0.000547,-0.009136,NaN
2025-04-22,-0.010075,-0.009833,-0.013003,-0.010075,NaN


In [19]:
all_data=pd.DataFrame()
all_data['close'] = sp500_data['Close']
all_data['close_diff'] = sp500_data_diff['Close']
all_data['usdeur'] = usd_to_euro_data['Close']
all_data['usdeur_diff'] = usd_to_euro_data_diff['Close']
all_data


,close,close_diff,usdeur,usdeur_diff
Date,,,,
2023-01-03,3824.139893,NaN,0.93653,0.002999
2023-01-04,3852.969971,0.007539,0.94815,0.012407
2023-01-05,3808.100098,-0.011646,0.94283,-0.005611
2023-01-06,3895.080078,0.022841,0.95037,0.007997
2023-01-09,3892.090088,-0.000768,0.93841,-0.012585
...,...,...,...,...
2025-04-21,5158.200195,-0.023567,NaN,NaN
2025-04-22,5287.759766,0.025117,0.86860,-0.010075
2025-04-23,5375.859863,0.016661,0.88096,0.014230


In [20]:
start_date = '2024-08-08'  # Define the start date
initial_eur = 33000  # Initial value in EUR

# Filter data starting from the defined start date
all_data_filtered = all_data.loc[start_date:].copy()

# Calculate the cumulative value in EUR
all_data_filtered['cumulative_eur'] = initial_eur * (1 + all_data_filtered['close_diff']).cumprod() * (1 + all_data_filtered['usdeur_diff']).cumprod() #/ all_data_filtered['usdeur'])
all_data_filtered.ffill(inplace=True)  # Forward fill to handle any NaN values
all_data_filtered['cumulative_eur_diff'] = all_data_filtered['cumulative_eur'].pct_change(fill_method=None)

# Add the cumulative_eur column back to the original all_data DataFrame
all_data_filtered

,close,close_diff,usdeur,usdeur_diff,cumulative_eur,cumulative_eur_diff
Date,,,,,,
2024-08-08,5319.310059,0.023043,0.91507,-0.000175,33754.504984,NaN
2024-08-09,5344.160156,0.004672,0.91611,0.001137,33950.736511,0.005813
2024-08-12,5344.390137,0.000043,0.91627,0.000175,33958.128775,0.000218
2024-08-13,5434.430176,0.016848,0.91439,-0.002052,34459.392388,0.014761
2024-08-14,5455.209961,0.003824,0.90934,-0.005523,34400.115323,-0.001720
...,...,...,...,...,...,...
2025-04-21,5158.200195,-0.023567,0.87744,-0.009136,32159.560236,0.000000
2025-04-22,5287.759766,0.025117,0.86860,-0.010075,31866.052465,-0.009127
2025-04-23,5375.859863,0.016661,0.88096,0.014230,32857.978652,0.031128


In [21]:
# Create a bar chart for cumulative_eur_diff
fig = px.bar(
    all_data_filtered,
    x=all_data_filtered.index,
    y='cumulative_eur_diff',
    title='Cumulative EUR Difference by Date',
    labels={'cumulative_eur_diff': 'Cumulative EUR Difference', 'index': 'Date'},
)

# Customize the layout
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Cumulative EUR Difference (%)",
    yaxis_tickformat="0.00%",  # Format y-axis as percentage
    height=800,
    width=1500,
    yaxis_zeroline=True,
    yaxis_zerolinecolor="black",
    yaxis_zerolinewidth=2,
)

# Add a line for cumulative_eur on the second y-axis
fig.add_trace(
    px.line(
        all_data_filtered,
        x=all_data_filtered.index,
        y='cumulative_eur',
    ).data[0].update(yaxis='y2',line=dict(color='orange'))  # Corrected the update method and closed the parentheses
)

# Update the layout to include a second y-axis
fig.update_layout(
    yaxis2=dict(
        title="Cumulative EUR",
        overlaying="y",
        side="right",
        showgrid=False,
        matches=None,
    )
)

fig.show()

In [22]:
# Create a bar chart for cumulative_eur_diff with a gradient color
fig = px.bar(
    all_data_filtered,
    x=all_data_filtered.index,
    y='cumulative_eur_diff',
    title='Cumulative EUR Difference by Date',
    labels={'cumulative_eur_diff': 'Cumulative EUR Difference', 'index': 'Date'},
    color='cumulative_eur_diff',  # Map color to the cumulative_eur_diff values
    color_continuous_scale='RdYlGn',  # Gradient from red to green
    color_continuous_midpoint=0,  # Set midpoint for the gradient
    range_color=[-0.05, 0.05],  # Set the range for color mapping
)

# Customize the layout
fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Cumulative EUR Difference (%)",
    yaxis_tickformat="0.00%",  # Format y-axis as percentage
    height=800,
    width=1500,
    yaxis_zeroline=True,
    yaxis_zerolinecolor="black",
    yaxis_zerolinewidth=2,
)

# Add a line for cumulative_eur on the second y-axis
fig.add_trace(
    px.line(
        all_data_filtered,
        x=all_data_filtered.index,
        y='cumulative_eur',
    ).data[0].update(yaxis='y2', line=dict(color='blue'))
)

# Update the layout to include a second y-axis
fig.update_layout(
    yaxis2=dict(
        title="Cumulative EUR",
        overlaying="y",
        side="right",
        showgrid=False,
        matches=None,
    )
)

fig.update_layout(
    annotations=[
        {
        'x': '2025-04-03',
        'y': 35500, 
        'xref': 'x', 
        'yref': 'y2',
        'text': 'tariffs', 'showarrow': True, 'arrowhead': 2, 'ax': 0, 'ay': -40},
    {
        'x': '2025-04-09',
        'y': 38000, 
        'xref': 'x', 
        'yref': 'y2',
        'text': 'tariffs removed', 'showarrow': True, 'arrowhead': 2, 'ax': 0, 'ay': -40},
    ]
)

fig.show()
